In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/AI/Datasets"


In [3]:
%cd /content/drive/MyDrive/AI/Datasets

/content/drive/MyDrive/AI/Datasets


In [4]:
!kaggle datasets download -d nightfury007/fercustomdataset-3classes

100% 83.1M/83.1M [00:05<00:00, 21.5MB/s]
100% 83.1M/83.1M [00:05<00:00, 17.0MB/s]


In [5]:
import zipfile
for file in os.listdir():
    if file.endswith(".zip"):
        with zipfile.ZipFile(file, "r") as zip_file:
            zip_file.extractall()
        os.remove(file)

In [6]:
import os
import torch
import torchvision
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torchvision.utils import make_grid
import matplotlib
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
import glob
import torch.optim as optim
import time
import cv2
import torchvision.datasets as datasets
%matplotlib inline

In [7]:
data_dir = '/content/drive/MyDrive/AI/Datasets/FER_Custom_Dataset'
IMAGE_DIM=256

print(os.listdir(data_dir))

['Disappointed', 'interested', 'neutral']


In [8]:
def load_data(path, test_split, batch_size, input_size, shuffle_dataset=True,random_seed=42):
    
    transform_dict ={'src': transforms.Compose([transforms.Resize(IMAGE_DIM), 
                                                transforms.RandomHorizontalFlip(), 
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                     std=[0.229, 0.224, 0.225]
                                                                    )
                                               ]
                                              )
                    }
    
    dataset = datasets.ImageFolder(data_dir, transform=transform_dict.get('src'))
    dataset_size = len(dataset)
    train_test_indices = list(range(dataset_size))
    test_split = int(np.floor(test_split * dataset_size))
    
    if shuffle_dataset :
        np.random.seed(random_seed)
        np.random.shuffle(train_test_indices)
    train_indices, test_indices = train_test_indices[test_split:], train_test_indices[:test_split]
    

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    test_sampler = SubsetRandomSampler(test_indices)

    data_loader_train = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                              sampler=train_sampler)
    data_loader_test = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                    sampler=test_sampler)
    
    return data_loader_train, data_loader_test

In [9]:
train_loader, test_loader=load_data(data_dir,0.2,32,(64,64))

print(len(train_loader), len(test_loader))

1096 274


In [10]:
######## Write your code here ##############
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=False)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
model.to(device)

num_epochs = 2
total_steps = len(train_loader)
t1 = time.time()
for epoch in range(num_epochs):
    for i, data in enumerate(train_loader):
        images, labels = data[0].to(device), data[1].to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backprop and optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Train accuracy
        total = labels.size(0)
        _,predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                    (correct / total) * 100))
            
print("######## Training Finished in {} seconds ###########".format(time.time()-t1))

Device: cuda:0
Epoch [1/2], Step [100/1096], Loss: 1.1894, Accuracy: 34.38%
Epoch [1/2], Step [200/1096], Loss: 1.0684, Accuracy: 37.50%
Epoch [1/2], Step [300/1096], Loss: 1.1106, Accuracy: 34.38%
Epoch [1/2], Step [400/1096], Loss: 1.0375, Accuracy: 46.88%
Epoch [1/2], Step [500/1096], Loss: 1.1250, Accuracy: 34.38%
Epoch [1/2], Step [600/1096], Loss: 1.1109, Accuracy: 46.88%
Epoch [1/2], Step [700/1096], Loss: 0.9926, Accuracy: 59.38%
Epoch [1/2], Step [800/1096], Loss: 1.0658, Accuracy: 50.00%
Epoch [1/2], Step [900/1096], Loss: 1.0106, Accuracy: 43.75%
Epoch [1/2], Step [1000/1096], Loss: 1.0716, Accuracy: 40.62%
Epoch [2/2], Step [100/1096], Loss: 1.1138, Accuracy: 40.62%
Epoch [2/2], Step [200/1096], Loss: 1.0021, Accuracy: 56.25%
Epoch [2/2], Step [300/1096], Loss: 0.9971, Accuracy: 56.25%
Epoch [2/2], Step [400/1096], Loss: 1.0733, Accuracy: 46.88%
Epoch [2/2], Step [500/1096], Loss: 1.1236, Accuracy: 31.25%
Epoch [2/2], Step [600/1096], Loss: 1.1963, Accuracy: 21.88%
Epoch [2